# Day 09. Exercise 00
# Regularization

## 0. Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn import metrics, svm, linear_model, tree, ensemble

import pickle


## 1. Preprocessing

1. Read the file `dayofweek.csv` that you used in the previous day to a dataframe.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/dayofweek.csv', index_col='index')
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
index,,,,,,,,,,,,,,,,,,,,,
0,-0.788667,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.756764,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.724861,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.692958,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.661055,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,-0.533442,0.945382,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,-0.629151,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,-0.597248,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
X = df.drop(columns='dayofweek')
y = df['dayofweek']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [5]:
y_train.value_counts() / y_train.shape[0] - y_test.value_counts() / y_test.shape[0]

3   -0.002265
6    0.001365
1   -0.000259
5    0.001216
2   -0.000478
0    0.000979
4   -0.000557
Name: dayofweek, dtype: float64

## 2. Logreg regularization

### a. Default regularization

1. Train a baseline model with the only parameters `random_state=21`, `fit_intercept=False`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model


The result of the code where you trained and evaluated the baseline model should be exactly like this (use `%%time` to get the info about how long it took to run the cell):

```
train -  0.62902   |   valid -  0.59259
train -  0.64633   |   valid -  0.62963
train -  0.63479   |   valid -  0.56296
train -  0.65622   |   valid -  0.61481
train -  0.63397   |   valid -  0.57778
train -  0.64056   |   valid -  0.59259
train -  0.64138   |   valid -  0.65926
train -  0.65952   |   valid -  0.56296
train -  0.64333   |   valid -  0.59701
train -  0.63674   |   valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
```

In [6]:
def cross_validation_test(model, split_n=10):
    kf = StratifiedKFold(split_n)
    X = X_train.values
    y = y_train.values

    for train, test in kf.split(X, y):
        model.fit(X[train], y[train])

        train_accuracy = metrics.accuracy_score(model.predict(X[train]), y[train])
        test_accuracy = metrics.accuracy_score(model.predict(X[test]), y[test])

        print(f'train -  {train_accuracy:1.5f}  |  valid -  {test_accuracy:1.5f}')

    accuracy_values = cross_val_score(model, X, y, cv=kf, n_jobs=-1)
    print(f'Average accuracy on crossval is {accuracy_values.mean():1.5f}')
    print(f'Std is {accuracy_values.std():1.5f}\n')

In [7]:
%%timeit -n 1 -r 1

logreg = linear_model.LogisticRegression(random_state=21, fit_intercept=False)
cross_validation_test(logreg, 10)

train -  0.62902  |  valid -  0.59259
train -  0.64633  |  valid -  0.62963
train -  0.63479  |  valid -  0.56296
train -  0.65622  |  valid -  0.61481
train -  0.63397  |  valid -  0.57778
train -  0.64056  |  valid -  0.59259
train -  0.64138  |  valid -  0.65926
train -  0.65952  |  valid -  0.56296
train -  0.64333  |  valid -  0.59701
train -  0.63674  |  valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943

1.74 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### b. Optimizing regularization parameters

1. In the cells below try different values of penalty: `none`, `l1`, `l2` – you can change the values of solver too.

In [8]:
%%timeit -n 1 -r 1

logreg = linear_model.LogisticRegression(random_state=21, fit_intercept=False, max_iter=1000, penalty='none')
cross_validation_test(logreg, 10)

train -  0.66694  |  valid -  0.63704
train -  0.65787  |  valid -  0.65926
train -  0.66694  |  valid -  0.57778
train -  0.66529  |  valid -  0.62963
train -  0.66694  |  valid -  0.62222
train -  0.65952  |  valid -  0.57778
train -  0.64963  |  valid -  0.69630
train -  0.68425  |  valid -  0.61481
train -  0.66474  |  valid -  0.62687
train -  0.65651  |  valid -  0.60448
Average accuracy on crossval is 0.62462
Std is 0.03379

2.92 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%%timeit -n 1 -r 1

logreg = linear_model.LogisticRegression(random_state=21, fit_intercept=False, max_iter=1000, penalty='l1', solver='liblinear')
cross_validation_test(logreg, 10)

train -  0.61830  |  valid -  0.54815
train -  0.62737  |  valid -  0.62222
train -  0.60511  |  valid -  0.54074
train -  0.63644  |  valid -  0.62222
train -  0.62407  |  valid -  0.55556
train -  0.62325  |  valid -  0.58519
train -  0.61253  |  valid -  0.63704
train -  0.64716  |  valid -  0.58519
train -  0.63015  |  valid -  0.59701
train -  0.61367  |  valid -  0.59701
Average accuracy on crossval is 0.58903
Std is 0.03129

191 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n 1 -r 1

logreg = linear_model.LogisticRegression(random_state=21, fit_intercept=False, max_iter=1000, penalty='l2', solver='liblinear')
cross_validation_test(logreg, 10)

train -  0.61006  |  valid -  0.56296
train -  0.61665  |  valid -  0.61481
train -  0.61336  |  valid -  0.59259
train -  0.62902  |  valid -  0.60741
train -  0.60923  |  valid -  0.55556
train -  0.61500  |  valid -  0.57778
train -  0.61665  |  valid -  0.61481
train -  0.64056  |  valid -  0.53333
train -  0.62109  |  valid -  0.58209
train -  0.61120  |  valid -  0.57463
Average accuracy on crossval is 0.58160
Std is 0.02532

121 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 3. SVM regularization

### a. Default regularization

1. Train a baseline model with the only parameters `probability=True`, `kernel='linear'`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [11]:
%%timeit -n 1 -r 1

svc = svm.SVC(probability=True, kernel='linear', random_state=21)
cross_validation_test(svc, 10)

train -  0.70486  |  valid -  0.65926
train -  0.69662  |  valid -  0.75556
train -  0.69415  |  valid -  0.62222
train -  0.70239  |  valid -  0.65185
train -  0.69085  |  valid -  0.65185
train -  0.68920  |  valid -  0.64444
train -  0.69250  |  valid -  0.72593
train -  0.70074  |  valid -  0.62222
train -  0.69605  |  valid -  0.61940
train -  0.71087  |  valid -  0.63433
Average accuracy on crossval is 0.65871
Std is 0.04359

4.61 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `C`.

In [12]:
%%timeit -n 1 -r 1

svc = svm.SVC(probability=True, kernel='linear', random_state=21, C=0.1)
cross_validation_test(svc, 10)

train -  0.58120  |  valid -  0.55556
train -  0.57543  |  valid -  0.56296
train -  0.57378  |  valid -  0.57037
train -  0.59275  |  valid -  0.57037
train -  0.58120  |  valid -  0.54815
train -  0.57955  |  valid -  0.54815
train -  0.57296  |  valid -  0.61481
train -  0.59192  |  valid -  0.54815
train -  0.59967  |  valid -  0.52985
train -  0.57825  |  valid -  0.57463
Average accuracy on crossval is 0.56230
Std is 0.02177

5.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n 1 -r 1

svc = svm.SVC(probability=True, kernel='linear', random_state=21, C=0.5)
cross_validation_test(svc, 10)

train -  0.66694  |  valid -  0.63704
train -  0.66612  |  valid -  0.73333
train -  0.67271  |  valid -  0.60741
train -  0.67354  |  valid -  0.62963
train -  0.67766  |  valid -  0.64444
train -  0.66529  |  valid -  0.61481
train -  0.66200  |  valid -  0.68889
train -  0.66529  |  valid -  0.57037
train -  0.67463  |  valid -  0.59701
train -  0.66804  |  valid -  0.61194
Average accuracy on crossval is 0.63349
Std is 0.04471

4.71 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
%%timeit -n 1 -r 1

svc = svm.SVC(probability=True, kernel='linear', random_state=21, C=1.5)
cross_validation_test(svc, 10)

train -  0.70734  |  valid -  0.65926
train -  0.70486  |  valid -  0.74815
train -  0.74361  |  valid -  0.64444
train -  0.70816  |  valid -  0.67407
train -  0.70239  |  valid -  0.67407
train -  0.69909  |  valid -  0.64444
train -  0.70239  |  valid -  0.72593
train -  0.70239  |  valid -  0.62963
train -  0.70840  |  valid -  0.64925
train -  0.71911  |  valid -  0.64179
Average accuracy on crossval is 0.66910
Std is 0.03679

4.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n 1 -r 1

svc = svm.SVC(probability=True, kernel='linear', random_state=21, C=2)
cross_validation_test(svc, 10)

train -  0.70734  |  valid -  0.65926
train -  0.71393  |  valid -  0.75556
train -  0.74526  |  valid -  0.63704
train -  0.71558  |  valid -  0.66667
train -  0.71146  |  valid -  0.67407
train -  0.70157  |  valid -  0.63704
train -  0.70651  |  valid -  0.71852
train -  0.70981  |  valid -  0.64444
train -  0.72405  |  valid -  0.64925
train -  0.72488  |  valid -  0.64179
Average accuracy on crossval is 0.66836
Std is 0.03721

4.77 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 4. Tree

### a. Default regularization

1. Train a baseline model with the only parameter `max_depth=10` and `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [16]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=10, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  0.81039  |  valid -  0.74074
train -  0.77741  |  valid -  0.74074
train -  0.83347  |  valid -  0.70370
train -  0.79720  |  valid -  0.76296
train -  0.82440  |  valid -  0.75556
train -  0.80379  |  valid -  0.68889
train -  0.80709  |  valid -  0.76296
train -  0.80132  |  valid -  0.65926
train -  0.80807  |  valid -  0.75373
train -  0.80478  |  valid -  0.68657
Average accuracy on crossval is 0.72551
Std is 0.03562

132 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `max_depth`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [17]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=1, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  0.35367  |  valid -  0.37037
train -  0.35449  |  valid -  0.36296
train -  0.35614  |  valid -  0.34815
train -  0.35449  |  valid -  0.36296
train -  0.35532  |  valid -  0.35556
train -  0.35367  |  valid -  0.37037
train -  0.35532  |  valid -  0.35556
train -  0.35614  |  valid -  0.34815
train -  0.35667  |  valid -  0.34328
train -  0.35750  |  valid -  0.33582
Average accuracy on crossval is 0.35532
Std is 0.01094

56.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=5, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  0.59522  |  valid -  0.53333
train -  0.56307  |  valid -  0.53333
train -  0.60181  |  valid -  0.55556
train -  0.59604  |  valid -  0.57037
train -  0.60264  |  valid -  0.57778
train -  0.57955  |  valid -  0.53333
train -  0.58368  |  valid -  0.54815
train -  0.59275  |  valid -  0.51111
train -  0.58237  |  valid -  0.56716
train -  0.60132  |  valid -  0.50000
Average accuracy on crossval is 0.54301
Std is 0.02423

67.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=15, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  0.95796  |  valid -  0.82222
train -  0.93075  |  valid -  0.83704
train -  0.95631  |  valid -  0.83704
train -  0.95301  |  valid -  0.86667
train -  0.95136  |  valid -  0.88889
train -  0.94724  |  valid -  0.82222
train -  0.95466  |  valid -  0.90370
train -  0.94971  |  valid -  0.87407
train -  0.95305  |  valid -  0.83582
train -  0.94316  |  valid -  0.85821
Average accuracy on crossval is 0.85459
Std is 0.02682

87.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=25, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  1.00000  |  valid -  0.85926
train -  1.00000  |  valid -  0.91852
train -  0.99918  |  valid -  0.86667
train -  1.00000  |  valid -  0.91111
train -  0.99918  |  valid -  0.88889
train -  0.99835  |  valid -  0.85185
train -  0.99753  |  valid -  0.92593
train -  1.00000  |  valid -  0.88148
train -  1.00000  |  valid -  0.88060
train -  1.00000  |  valid -  0.88060
Average accuracy on crossval is 0.88649
Std is 0.02371

86 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:
%%timeit -n 1 -r 1

tree_classifier = tree.DecisionTreeClassifier(max_depth=50, random_state=21)
cross_validation_test(tree_classifier, 10)

train -  1.00000  |  valid -  0.85926
train -  1.00000  |  valid -  0.91852
train -  1.00000  |  valid -  0.86667
train -  1.00000  |  valid -  0.91111
train -  1.00000  |  valid -  0.88148
train -  1.00000  |  valid -  0.85185
train -  1.00000  |  valid -  0.92593
train -  1.00000  |  valid -  0.88148
train -  1.00000  |  valid -  0.88060
train -  1.00000  |  valid -  0.88060
Average accuracy on crossval is 0.88575
Std is 0.02374

90.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 5. Random forest

### a. Default regularization

1. Train a baseline model with the only parameters `n_estimators=50`, `max_depth=14`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [22]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)
cross_validation_test(random_forest, 10)

train -  0.96455  |  valid -  0.88148
train -  0.96208  |  valid -  0.91852
train -  0.96785  |  valid -  0.86667
train -  0.96455  |  valid -  0.89630
train -  0.96538  |  valid -  0.91111
train -  0.96538  |  valid -  0.88148
train -  0.97115  |  valid -  0.91852
train -  0.96867  |  valid -  0.85185
train -  0.97364  |  valid -  0.88060
train -  0.97941  |  valid -  0.86567
Average accuracy on crossval is 0.88722
Std is 0.02204

1.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### b. Optimizing regularization parameters

1. In the new cells try different values of the parameters `max_depth` and `n_estimators`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [23]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=10, max_depth=14, random_state=21)
cross_validation_test(random_forest, 10)

train -  0.95548  |  valid -  0.85926
train -  0.93157  |  valid -  0.88148
train -  0.95960  |  valid -  0.85926
train -  0.92910  |  valid -  0.87407
train -  0.95136  |  valid -  0.89630
train -  0.94889  |  valid -  0.84444
train -  0.93405  |  valid -  0.85185
train -  0.93899  |  valid -  0.83704
train -  0.95552  |  valid -  0.87313
train -  0.95387  |  valid -  0.85075
Average accuracy on crossval is 0.86276
Std is 0.01731

369 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [24]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=150, max_depth=14, random_state=21)
cross_validation_test(random_forest, 10)

train -  0.96702  |  valid -  0.87407
train -  0.96620  |  valid -  0.91852
train -  0.97115  |  valid -  0.88148
train -  0.96950  |  valid -  0.88889
train -  0.96785  |  valid -  0.89630
train -  0.97032  |  valid -  0.86667
train -  0.97197  |  valid -  0.91111
train -  0.97032  |  valid -  0.88889
train -  0.97446  |  valid -  0.88060
train -  0.97694  |  valid -  0.85075
Average accuracy on crossval is 0.88573
Std is 0.01902

3.79 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=50, max_depth=5, random_state=21)
cross_validation_test(random_forest, 10)

train -  0.62984  |  valid -  0.58519
train -  0.57791  |  valid -  0.55556
train -  0.60181  |  valid -  0.57778
train -  0.59522  |  valid -  0.59259
train -  0.61748  |  valid -  0.57778
train -  0.59687  |  valid -  0.57778
train -  0.61500  |  valid -  0.60741
train -  0.59522  |  valid -  0.57037
train -  0.59967  |  valid -  0.57463
train -  0.60297  |  valid -  0.54478
Average accuracy on crossval is 0.57638
Std is 0.01668

1.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=50, max_depth=35, random_state=21)
cross_validation_test(random_forest, 10)

train -  1.00000  |  valid -  0.89630
train -  1.00000  |  valid -  0.94074
train -  1.00000  |  valid -  0.90370
train -  1.00000  |  valid -  0.93333
train -  1.00000  |  valid -  0.90370
train -  1.00000  |  valid -  0.89630
train -  1.00000  |  valid -  0.91852
train -  1.00000  |  valid -  0.89630
train -  1.00000  |  valid -  0.94030
train -  0.99918  |  valid -  0.89552
Average accuracy on crossval is 0.91247
Std is 0.01809

1.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=150, max_depth=35, random_state=21)
cross_validation_test(random_forest, 10)

train -  1.00000  |  valid -  0.90370
train -  1.00000  |  valid -  0.94815
train -  1.00000  |  valid -  0.90370
train -  1.00000  |  valid -  0.94074
train -  1.00000  |  valid -  0.91111
train -  1.00000  |  valid -  0.89630
train -  1.00000  |  valid -  0.92593
train -  1.00000  |  valid -  0.90370
train -  1.00000  |  valid -  0.92537
train -  1.00000  |  valid -  0.89552
Average accuracy on crossval is 0.91542
Std is 0.01765

4.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:
%%timeit -n 1 -r 1

random_forest = ensemble.RandomForestClassifier(n_estimators=100, max_depth=25, random_state=21)
cross_validation_test(random_forest, 10)

train -  0.99918  |  valid -  0.90370
train -  0.99918  |  valid -  0.96296
train -  0.99918  |  valid -  0.89630
train -  1.00000  |  valid -  0.94074
train -  0.99918  |  valid -  0.91852
train -  0.99835  |  valid -  0.89630
train -  0.99918  |  valid -  0.92593
train -  0.99918  |  valid -  0.89630
train -  1.00000  |  valid -  0.93284
train -  1.00000  |  valid -  0.90299
Average accuracy on crossval is 0.91766
Std is 0.02160

2.71 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## 6. Predictions

### 1. Choose the best model and use it to make predictions for the test dataset.

In [29]:
random_forest = ensemble.RandomForestClassifier(n_estimators=100, max_depth=25, random_state=21)
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=25, random_state=21)

### 2. Calculate the final accuracy.

In [30]:
y_pred = random_forest.predict(X_test)

print('Total accuracy:', metrics.accuracy_score(y_pred, y_test))

Total accuracy: 0.9289940828402367


### 3. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your test dataset).

In [31]:
most_wrong_predicted = y_test[y_test != y_pred].value_counts().reset_index().iloc[0]
print(f'Most wrong predicted day: {most_wrong_predicted["index"]}')


Most wrong predicted day: 0


In [32]:
total_errors_frequency = most_wrong_predicted["dayofweek"] / y_test.value_counts()[most_wrong_predicted["index"]] * 100
print(f'Total percentage of errors in predicting this day: {total_errors_frequency}')

Total percentage of errors in predicting this day: 25.925925925925924


### 4. Save the model.


In [33]:
with open('../data/random_forest_classifier.pkl', 'wb') as fout:
    pickle.dump(random_forest, fout)

In [34]:
%ls ../data/random_forest_classifier.pkl

../data/random_forest_classifier.pkl
